# Advent of code - day 7
## Problem
Simple directories

In [ ]:
// See https://aka.ms/new-console-template for more information

using System.Collections;
using System.Runtime.InteropServices.ComTypes;

var input = System.IO.File.ReadAllLines("input.txt");

In [ ]:
var result = new MyDirectory("\\", null);

var currentDir  = result;

foreach (var line in input.Skip(1))
{
    var split = line.Split(' ');
    switch (split[0])
    {
        case "$":
            if (split[1] == "cd")
            {
                if (split[2] == "..")
                {
                    currentDir = currentDir.Parent;
                    continue;
                }
                
                currentDir = currentDir.Items
                    .Single(x => x.Name == split[2] 
                                 && x is MyDirectory) as MyDirectory;
            }
        break;
        case "dir":
            var newDir = new MyDirectory(split[1], currentDir);
            currentDir.Items.Add(newDir);
            break;
        default:
            var newFile = new MyFile(split[1], long.Parse(split[0]));
            currentDir.Items.Add(newFile);
            break;
    }
}

var totals = new List<long>();

getAllDirectorySizes(result);

var unusedSpace = 70000000 - result.GetSize();

var total = totals.OrderBy(x => x).SkipWhile(x => x + unusedSpace < 30000000).First();

Console.Read();

void getAllDirectorySizes(MyDirectory directory)
{
    totals.Add(directory.GetSize());
    foreach (var dir in directory.Items.Where(x => x is MyDirectory))
    {
        getAllDirectorySizes((MyDirectory)dir);
    }
}

void GetSmallDirectories(MyDirectory directory)
{
    long totalSize = 0;
    foreach (var dir in directory.Items.Where(x => x is MyDirectory))
    {
        var dirSize = ((MyDirectory)dir).GetSize();
        if (dirSize < 100000)
        {
            totalSize += dirSize;
        }

        if ((dir as MyDirectory).Items.Any(x => x is MyDirectory))
        {
            GetSmallDirectories((MyDirectory)dir);
        }
    }
    totals.Add(totalSize);
}

public abstract class MyItem
{
    public string Name { get; set; }
}

public class MyDirectory : MyItem
{
    public MyDirectory(string n, MyDirectory p)
    {
        Name = n;
        Parent = p;
        Items = new List<MyItem>();
    }
    public MyDirectory Parent { get; set; }
    public List<MyItem> Items { get; set; }

    public long GetSize()
    {
        var files = Items.Where(x => x is MyFile).Sum(x => (x as MyFile).Size);
        var directories = Items.Where(x => x is MyDirectory).Sum(x => (x as MyDirectory).GetSize());

        return files + directories;
    }
}

public class MyFile : MyItem
{
    public MyFile(string n, long s)
    {
        Name = n;
        Size = s;
    }
    public long Size { get; set; }
}